In [6]:
import requests
import json
from bs4 import BeautifulSoup
import re

yelp_api_key = 'skLkCn-T3n7MChhfKDzQn4dy0T2uefMBGPtauGVQ0U86oF1WVs8jnaM-pOtdgamu1fQysI8fROfPHsHlZUOZ6brQNSiqmGKC81Ta5W1S_Mxx6jYjuJAZg1nRc2Q8ZXYx'

def get_yelp_restaurant_data(yelp_api_key, location, categories, limit=1500):
    cache_file = 'yelp_cache.json'
    try:
        with open(cache_file, 'r') as file:
            yelp_data = json.load(file)
            return yelp_data
    except FileNotFoundError:
        yelp_data = {}

    endpoint = 'https://api.yelp.com/v3/businesses/search'
    headers = {
        'Authorization': f'Bearer {yelp_api_key}',
    }

    offset = 0

    try:
        while offset < limit:
            params = {
                'location': location,
                'categories': categories,
                'limit': min(50, limit - offset),
                'offset': offset,
            }

            response = requests.get(endpoint, headers=headers, params=params)
            data = response.json()

            businesses = data.get('businesses', [])
            for business in businesses:
                yelp_data[business['id']] = business

            if len(businesses) < min(50, limit - offset):
                break

            offset += 50

            if offset >= 1000:
                break

        with open(cache_file, 'w') as file:
            json.dump(yelp_data, file)

    except Exception as e:
        print(f"An error occurred: {e}")

    return yelp_data

location = 'Michigan'
categories = 'restaurants'
yelp_data = get_yelp_restaurant_data(yelp_api_key, location, categories, limit=1500)

def get_yelp_reviews(yelp_api_key, business_id, yelp_data):
    if 'reviews' in yelp_data[business_id]:
        return yelp_data[business_id]['reviews']

    endpoint = f'https://api.yelp.com/v3/businesses/{business_id}/reviews'
    headers = {
        'Authorization': f'Bearer {yelp_api_key}',
    }

    try:
        response = requests.get(endpoint, headers=headers)
        data = response.json()
        reviews = data.get('reviews', [])
        yelp_data[business_id]['reviews'] = reviews  # Cache the reviews
        return reviews
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


for business_id in yelp_data.keys():
    reviews = get_yelp_reviews(yelp_api_key, business_id, yelp_data)
    if reviews:
        yelp_data[business_id]['reviews'] = reviews

cache_file = 'yelp_cache.json'
with open(cache_file, 'w') as file:
    json.dump(yelp_data, file)

In [2]:
len(yelp_data)

1000

In [7]:
yelp_data

{'Xm2A4Xyd-dw4x86qZCV3hw': {'id': 'Xm2A4Xyd-dw4x86qZCV3hw',
  'alias': 'chartreuse-detroit',
  'name': 'Chartreuse',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/MMT1UOcfb44ZKiw8_fOhGw/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/chartreuse-detroit?adjust_creative=-MqpFu1Q404tRoOKumywUw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=-MqpFu1Q404tRoOKumywUw',
  'review_count': 832,
  'categories': [{'alias': 'newamerican', 'title': 'New American'},
   {'alias': 'tapasmallplates', 'title': 'Tapas/Small Plates'},
   {'alias': 'cocktailbars', 'title': 'Cocktail Bars'}],
  'rating': 4.5,
  'coordinates': {'latitude': 42.360517, 'longitude': -83.066075},
  'transactions': ['delivery'],
  'price': '$$$',
  'location': {'address1': '15 E Kirby St',
   'address2': 'Ste D',
   'address3': '',
   'city': 'Detroit',
   'zip_code': '48202',
   'country': 'US',
   'state': 'MI',
   'display_address': ['15 E Kirby St', 'Ste D', 'Detroit, MI 48202']